In [90]:
import pandas as pd
import os

# excel stuff doesn't work without this:
import sys
sys.path.append("C:/Users/shrey/anaconda3/Lib/site-packages")
# if someone else is using this replace the above path with the base dir of 'openpyxl'.

In [91]:
# read in main csv file
signups = pd.read_excel(os.path.join("data_input", "signups.xlsx"))

# filter by done/not done
signups = signups.loc[pd.isna(signups['Emailed'])]
signups.reset_index(drop = True, inplace=True)

# remove irrelevant/already done requests
signups = signups[["Ready",
                   "First Name",
                   "Last Name",
                   "Email",
                   "Phone",
                   "Are you part of a volunteer group?",
                   "Group Name",
                   "How many postcards are you requesting?",
                   "Comments / Feedback"]]

# strip dangling spaces in all columns
signups.columns = signups.columns.str.strip()

# remove duplicate email rows
signups.drop_duplicates(subset=["Email"], keep='first', inplace=True)
signups.reset_index(drop=True, inplace=True)

signups

,Ready,First Name,Last Name,Email,Phone,Are you part of a volunteer group?,Group Name,How many postcards are you requesting?,Comments / Feedback
0,NaN,Roxi,Zilmer,roxnhr@gmail.com,3105038518,Yes,Mighty Monday Writers,500,thank you!
1,NaN,Pam,Kamps,pamkamps@gmail.com,7142711742,NaN,NaN,500,NaN
2,NaN,Loretta,Donelan,loretta.donelan@gmail.com,6039330299,NaN,NaN,200,NaN
3,NaN,Catherine,Rubenstein,carube7@yahoo.com,5627600955,NaN,NaN,200,NaN
4,NaN,Judith,Golden,jgandelgolden@gmail.com,310-869-9032,Yes,Grandparents for Democracy,100,NaN
...,...,...,...,...,...,...,...,...,...
157,NaN,Sherrie,Pasarell,slieberpasarell@gmail.com,8452359258,Yes,I volunteer with postcardstovoters,25,Thanks⚖️🗽
158,NaN,Sarah,Weissman,Sarahb579@aol.com,8473410459,Yes,FB - related to Georgia and writing,100,Happy to help! I just worked hard with phone b...
159,NaN,Roselee,Packham,Rozelee@msn.com,3106257673,Yes,Grandparents for democracy,50,NaN
160,NaN,Winona,Scheff,winonascheff@gmail.com,8604713993,NaN,NaN,50,NaN


In [92]:
import numpy as np
flagged_indices = []
over_400 = []

# flag rows with invalid postcard numbers
for row in signups.iterrows():
    idx = row[0]
    row = row[1]
    num_cards = row["How many postcards are you requesting?"]
    if not isinstance(num_cards, int):
        flagged_indices.append(idx)
    else:
        if num_cards >= 400:
            over_400.append(idx)

if (not os.path.exists("data_output")):
    os.mkdir("data_output")

# save flagged rows
signup_flags = signups.iloc[flagged_indices]
if os.path.isfile("data_output/flags.xlsx"):
    os.remove("data_output/flags.xlsx")

# save over 400 rows
signup_400 = signups.iloc[over_400]
if os.path.isfile("data_output/over_400.xlsx"):
    os.remove("data_output/over_400.xlsx")

# save rows that have comments
signup_comments = signups.loc[signups['Comments / Feedback'].notna()]
if os.path.isfile("data_output/comments.xlsx"):
    os.remove("data_output/comments.xlsx")

# save groups too bc y not
signup_groups = signups.loc[signups['Group Name'].notna()]
if os.path.isfile("data_output/groups.xlsx"):
    os.remove("data_output/groups.xlsx")

# save all as excel
signup_flags.to_excel(os.path.join("data_output", "flags.xlsx"), index=False)
signup_400.to_excel(os.path.join("data_output", "over_400.xlsx"), index=False)
signup_comments.to_excel(os.path.join("data_output", "comments.xlsx"), index=False)
signup_groups.to_excel(os.path.join("data_output", "groups.xlsx"), index=False)

# remove from signups dataframe
signups.drop(flagged_indices, inplace = True)
signups.drop(over_400, inplace = True)
signups.reset_index(drop=True, inplace=True)

print("Invalid postcard number: ", len(signup_flags), " rows")
print("Over 400: ", len(over_400), " rows")

Invalid postcard number:  2  rows
Over 400:  2  rows


In [93]:
# final preprocessing signups:
signups = signups[["First Name", "Last Name", "Email", "How many postcards are you requesting?"]]
signups

,First Name,Last Name,Email,How many postcards are you requesting?
0,Loretta,Donelan,loretta.donelan@gmail.com,200
1,Catherine,Rubenstein,carube7@yahoo.com,200
2,Judith,Golden,jgandelgolden@gmail.com,100
3,Alice,Stanley,Alicel.stanley@gmail.com,100
4,Mary,Norvell,norvellm@me.com,25
...,...,...,...,...
153,Sherrie,Pasarell,slieberpasarell@gmail.com,25
154,Sarah,Weissman,Sarahb579@aol.com,100
155,Roselee,Packham,Rozelee@msn.com,50
156,Winona,Scheff,winonascheff@gmail.com,50


In [94]:
# read in addresses csv file
addresses = pd.read_excel(os.path.join("data_input", "addresses.xlsx"))

# filter by done/not done
addresses = addresses.loc[addresses['Assigned'].isna()]
addresses = addresses[["Name", "Address 1", "Address 2"]]
addresses.reset_index(drop = True, inplace=True)
addresses.insert(0, "Group/volunteer", "")
addresses.insert(0, "Assigned", "")

print("length: ", len(addresses))
display(addresses.head())

length:  10005


,Assigned,Group/volunteer,Name,Address 1,Address 2
0,,,Rebecca Price,1250 Sweet Woods Dr,Lawrenceville GA 30044
1,,,Boris and Raisa Smychkovich,1250 Timberline Pl,Alpharetta GA 30005
2,,,Brenda Pope and Shalonda Scott,1250 Tommy Lee Cook Rd,Palmetto GA 30268
3,,,Barbara Giametta,1250 Village Terrace Ct,Atlanta GA 30338
4,,,Michelle Trader,1250 Virginia Ct NE,Atlanta GA 30306


In [95]:
# compute how many people can be mailed

running_card_num = 0
total_card_num = len(addresses)
total_needed = sum(signups["How many postcards are you requesting?"])
breaked = False

for row in signups.iterrows():
    idx = row[0]
    row = row[1]
    num_cards = row["How many postcards are you requesting?"]
    running_card_num += num_cards
    # if sum > total, then go back an index and break --> go back bc this one is too much
    if (running_card_num > total_card_num):
        idx -=1
        breaked = True
        break;

print("total addresses needed: ", total_needed)
print("we have: ", total_card_num)
if total_needed <= total_card_num:
    print("leftover: ", total_card_num - total_needed)

if breaked:
    print("Not enough names for all volunteers. Only ", idx+1, " will be sent.")

total addresses needed:  13090
we have:  10005
Not enough names for all volunteers. Only  119  will be sent.


In [99]:
# sort
if not os.path.exists(os.path.join("data_output", "sorted_addresses")):
    os.mkdir(os.path.join("data_output", "sorted_addresses"))

import os
curr_index = 0
for row in signups.iterrows():
    idx = row[0]
    row = row[1]
    fname = row['First Name']
    lname = row['Last Name']
    email = row['Email']
    fullname = fname.strip() + " " + lname.strip()
    num_cards = row['How many postcards are you requesting?']
    if curr_index + num_cards > len(addresses):
        print("Not enough addresses. Stopping.")
        break;
        
    personCards = addresses.iloc[curr_index: curr_index+num_cards]
    for i in range(num_cards):
        addresses.loc[curr_index, 'Assigned'] = "SJ"
        addresses.loc[curr_index, 'Group/volunteer'] = fullname
        curr_index += 1
    signups.at[idx, "Ready"] = "SJ"
    personCards = personCards.drop(columns=['Assigned', "Group/volunteer"])
    personCards = pd.concat([pd.DataFrame({'Name': [email]}), personCards])
    personCards.to_excel(os.path.join("data_output", "sorted_addresses", fullname + " " + str(num_cards) + ".xlsx"), index=False)
    
    if (idx % (len(signups)//10)) == 0:
        print(100*idx/len(signups), "%", "Done")

# replace all ""s in "Assigned" to NaNs
# addresses["Assigned"].loc[addresses['Assigned'] == "", ] = np.nan

addresses["Assigned"].loc[addresses['Assigned'] == ""] = np.nan

addresses.to_excel(os.path.join("data_output", "addresses.xlsx"))
display(addresses)
signups.to_excel(os.path.join("data_output", "signups.xlsx"))

0.0 % Done
9.49367088607595 % Done
18.9873417721519 % Done
28.481012658227847 % Done
37.9746835443038 % Done
47.46835443037975 % Done
56.962025316455694 % Done
66.45569620253164 % Done
Not enough addresses. Stopping.


,Assigned,Group/volunteer,Name,Address 1,Address 2
0,SJ,Loretta Donelan,Rebecca Price,1250 Sweet Woods Dr,Lawrenceville GA 30044
1,SJ,Loretta Donelan,Boris and Raisa Smychkovich,1250 Timberline Pl,Alpharetta GA 30005
2,SJ,Loretta Donelan,Brenda Pope and Shalonda Scott,1250 Tommy Lee Cook Rd,Palmetto GA 30268
3,SJ,Loretta Donelan,Barbara Giametta,1250 Village Terrace Ct,Atlanta GA 30338
4,SJ,Loretta Donelan,Michelle Trader,1250 Virginia Ct NE,Atlanta GA 30306
...,...,...,...,...,...
10000,NaN,,Georglyn Joseph Williams,5757 Wellborn Creek Dr,Lithonia GA 30058
10001,NaN,,Ligia Su Gan,9007 Brixworth Pl NE,Brookhaven GA 30319
10002,NaN,,Logan Flesch,2687 Imperial Hills Dr,Tucker GA 30084
10003,NaN,,Valerie Winder,110 Custer Ave Lot 49,Eatonton GA 31024


In [100]:
signups

,First Name,Last Name,Email,How many postcards are you requesting?,Ready
0,Loretta,Donelan,loretta.donelan@gmail.com,200,SJ
1,Catherine,Rubenstein,carube7@yahoo.com,200,SJ
2,Judith,Golden,jgandelgolden@gmail.com,100,SJ
3,Alice,Stanley,Alicel.stanley@gmail.com,100,SJ
4,Mary,Norvell,norvellm@me.com,25,SJ
...,...,...,...,...,...
153,Sherrie,Pasarell,slieberpasarell@gmail.com,25,NaN
154,Sarah,Weissman,Sarahb579@aol.com,100,NaN
155,Roselee,Packham,Rozelee@msn.com,50,NaN
156,Winona,Scheff,winonascheff@gmail.com,50,NaN
